In [1]:
import xlrd
import pandas as pd
import re
import datetime

In [2]:
c_shops = {'США': ['6РМ', 'AMAZON', 'МК', 'СК', 'ASHFORD', 'AX', 'BF', 'BR', 'CARTERS', 'CHP', 'CK', 'COACH', 'DISNEY',
                   'DKNY', 'DOPESNOW', 'GAP_OLD', 'GF', 'GF-BF', 'GF-BR', 'GYM', 'JJ', 'JOULES', 'KATESPADE', 'KL',
                   'LEVIS', 'LD', 'MACYS', 'MK', 'NB', 'NORD', 'OLD', 'OLD-BANANA', 'OLD-GAP', 'OLD(GAP) ', 'RALPH',
                   'RL', 'SAKS', 'SHOPDIABETS', 'TIMB', 'TOMMY', 'USPOLO', 'VIVAIA', 'VS', '6PM', '6PM', 'UGG'],
           'Германия': ['СА', 'FAMILY', 'AVARCA', 'BS', 'CA', 'DM', 'PINKO', 'PROSHOP', 'SOFTSHELL', 'WORLDOFSWEETS',
                        'ZALANDO'],
           'Англия': ['AMAZON_UK', 'ASDA', 'M&S', 'MATALAN', 'TU'],
           'Португалия': ['CORTEFIEL', 'DESIGUAL', 'MASSIMO', 'WS'],
           'Россия': ['Авторские адвент-календари', 'Авторский дизайн от Captains Mate', 'Арахисовые пасты',
                      'Ароматный кофе', 'Банты ручной работы', 'Белевские сладости', 'Бомбочки для ванны',
                      'Восточные сладости', 'Домашняя одежда', 'Есть в наличии', 'Настя и Никита', 'Издательство МИФ',
                      'Нигма', 'Стрекоза', 'ARCA', 'Мелик-Пашаев', 'Пешком в историю', 'Поляндрия',
                      'Корейская косметика', 'Косметика Корея', 'Косметика Green Era', 'G.LOVE', 'SmoRodina',
                      'Сокровища пиратов', 'Необычный чай', 'Пазлы DaVici', 'Uzelki', 'Шанти Пунти', 'Polezzno',
                      'Сарапульский кондитер', 'Laboratorium', 'Сибирский кедр', 'TRAWA', 'Термосы KLEAN KANTEEN',
                      'Huggy', 'Шанти-Пунти', 'AIM', 'Crockid', 'Estel', 'LOCAL', 'Издательство Робинс',
                      'Развивающие игрушки Djeco', 'CONTE колготки', 'OPTMOYO', 'MODA', 'Pelican', 'PELICAN', 'LANOTTI',
                      'HAPPY', 'КОРЕЯ', 'Развивающие игры Djeco']}

month_dict = {'января': '01', 'февраля': '02', 'марта': '03', 'апреля': '04', 'мая': '05', 'июня': '06',
              'июля': '7', 'августа': '8', 'сентября': '9', 'октября': '10', 'ноября': '11', 'декабря': '12',
              'дек': '12'}

In [3]:
sv = {'MK': 'МК', 'CK': 'СК', 'CA': 'СА'}
temp = dict()
for k, v in sv.items():
    temp[v] = k
sv = sv | temp
sv

{'MK': 'МК', 'CK': 'СК', 'CA': 'СА', 'МК': 'MK', 'СК': 'CK', 'СА': 'CA'}

In [4]:
shop_cs = {}
for k, v in c_shops.items():
    for x in v:
        shop_cs[x] = k
shop_cs

{'6РМ': 'США',
 'AMAZON': 'США',
 'МК': 'США',
 'СК': 'США',
 'ASHFORD': 'США',
 'AX': 'США',
 'BF': 'США',
 'BR': 'США',
 'CARTERS': 'США',
 'CHP': 'США',
 'CK': 'США',
 'COACH': 'США',
 'DISNEY': 'США',
 'DKNY': 'США',
 'DOPESNOW': 'США',
 'GAP_OLD': 'США',
 'GF': 'США',
 'GF-BF': 'США',
 'GF-BR': 'США',
 'GYM': 'США',
 'JJ': 'США',
 'JOULES': 'США',
 'KATESPADE': 'США',
 'KL': 'США',
 'LEVIS': 'США',
 'LD': 'США',
 'MACYS': 'США',
 'MK': 'США',
 'NB': 'США',
 'NORD': 'США',
 'OLD': 'США',
 'OLD-BANANA': 'США',
 'OLD-GAP': 'США',
 'OLD(GAP) ': 'США',
 'RALPH': 'США',
 'RL': 'США',
 'SAKS': 'США',
 'SHOPDIABETS': 'США',
 'TIMB': 'США',
 'TOMMY': 'США',
 'USPOLO': 'США',
 'VIVAIA': 'США',
 'VS': 'США',
 '6PM': 'США',
 'UGG': 'США',
 'СА': 'Германия',
 'FAMILY': 'Германия',
 'AVARCA': 'Германия',
 'BS': 'Германия',
 'CA': 'Германия',
 'DM': 'Германия',
 'PINKO': 'Германия',
 'PROSHOP': 'Германия',
 'SOFTSHELL': 'Германия',
 'WORLDOFSWEETS': 'Германия',
 'ZALANDO': 'Германия',
 'AMAZON_U

In [5]:
def find_date(date_cell):
    words = ''
    try:
        words = re.split('\W+', date_cell.lower())
    except:
        print('Hello error')
    global month_dict
    months = month_dict.keys()
    mindex = -1
    for month in months:
        if month in words:
            mindex = words.index(month)
            break
    if mindex <= 0:
        raise ValueError
    result = words[mindex-1].rjust(2, '0') + '.' + month_dict[words[mindex]].rjust(2, '0') + '.21'
    return result

In [6]:
book = xlrd.open_workbook("D:\\job\\hidream\\TZ.xls")
print("The number of worksheets is {0}".format(book.nsheets))
print("Worksheet name(s): {0}".format(book.sheet_names()))
sh = book.sheet_by_index(2)
rs = book.sheet_by_index(3)
report = book.sheet_by_index(1)
print("{0} {1} {2}".format(sh.name, sh.nrows, sh.ncols))
print("Cell A2 is {0}".format(sh.cell_value(rowx=3, colx=0)))

The number of worksheets is 4
Worksheet name(s): ['result', 'Report', 'Base', 'rs']
Base 4809 9
Cell A2 is Ботинки Timberland рр 7


In [7]:
code_errs = []
avdate_errs = []

In [8]:
def find_code(name, product):
    global report
    global code_errs
    global c_shops
    global shop_cs
    name_two = ' '.join(name.split()[::-1])
    err_cod = 'None'
    for row in range(report.nrows):
        if report.cell_value(rowx=row, colx=3) == 'Секреты Стихий' and (name == 'Аня Шанскова' or name_two == 'Аня Шанскова') and product == 'Секреты Стихий':
            rest = name_two in report.cell_value(rowx=row, colx=1) or name in report.cell_value(rowx=row, colx=1)
            print('This result is', rest)
            print((name.lower() in report.cell_value(rowx=row, colx=1).lower()
             or name_two.lower() in report.cell_value(rowx=row, colx=1).lower())
            and report.cell_value(rowx=row, colx=3) == product)
        if ((name.lower() in report.cell_value(rowx=row, colx=1).lower()
             or name_two.lower() in report.cell_value(rowx=row, colx=1).lower())
            and report.cell_value(rowx=row, colx=3) == product):
            code = report.cell_value(rowx=row, colx=0)
            err_cod = code
            if 'доставка' in code.lower():
                for country in c_shops.keys():
                    if country.lower() in code.lower():
                        return [code, country]
            elif 'отправка' in code.lower():
                return [code, 'Россия']
            else:
                if product == 'Секреты Стихий':
                    print('Hi man')
                for shop in shop_cs.keys():
                    if shop.lower() in code.lower():
                        return [code, shop_cs[shop]]
    code_errs.append([name, product, name_two, err_cod])
    return None, None

In [9]:
def find_avdate(name, code):
    global rs
    global avdate_errs
    name_two = ' '.join(name.split()[::-1])
    for row in range(rs.nrows):
        if code is not None:
            for key in sv.keys():
                if key in code:
                    temp = code.replace(key, sv[key])
                    if ((name.lower() in rs.cell_value(rowx=row, colx=4).lower()
                         or name_two.lower() in rs.cell_value(rowx=row, colx=4).lower())
                        and (code.lower() in rs.cell_value(rowx=row, colx=5).lower()
                        or temp.lower() in rs.cell_value(rowx=row, colx=5).lower())):
                        # дата аванса, сумма аванса
                        dater = rs.cell_value(rowx=row, colx=0)
                        #print(row, 'last: ', dater)
                        dater = datetime.datetime(*xlrd.xldate_as_tuple(dater, book.datemode))
                        dater = dater.strftime("%d.%m.%y")
                        avsum = rs.cell_value(rowx=row, colx=1)
                        # print(dater, avsum)
                        return dater, avsum
                    break

            if ((name.lower() in rs.cell_value(rowx=row, colx=4).lower()
                 or name_two.lower() in rs.cell_value(rowx=row, colx=4).lower())
                and (code.lower() in rs.cell_value(rowx=row, colx=5).lower())):
                # дата аванса, сумма аванса
                dater = rs.cell_value(rowx=row, colx=0)
                #print(row, 'last here: ', dater)
                dater = datetime.datetime(*xlrd.xldate_as_tuple(dater, book.datemode))
                dater = dater.strftime("%d.%m.%y")
                avsum = rs.cell_value(rowx=row, colx=1)
                # print(dater, avsum)
                return dater, avsum

    avdate_errs.append([name, code])
    return None, None

In [10]:
output = pd.DataFrame(columns=['ID', 'Название', 'Направление', 'Контакт', 'Товар', 'Цена за 1 шт', 'Количество',
                               'Агентское вознаграждение', 'Итого', 'Стадия сделки', 'Код посылки наш',
                               'Суммарный вес', 'Страна', 'Код посылки склада', 'Дата аванса', 'Сумма аванса',
                               'Выдали покупку', 'Код покупки'])

deal_id = 135
direction = 'Общее'
contact = ''
product_name = ''
price = 0
quantity = 1
agent = 0
total = 0
status = 'Сделка успешна'
our_pc = 1
total_weight = 1
country = 'None'
wh_pc = 1
avdate = ''
avsum = 0
date = ''
code = ''
deal_name = ''

In [11]:
date_q = False
good_time = False
prod_id = 0

In [12]:
for row in range(sh.nrows):
    cell = sh.cell_value(rowx=row, colx=0)
    if type(cell) is str:
        if len(cell.split(' ')) == 2 and sh.cell_value(rowx=row, colx=1) == '':
            deal_id += 1
            contact = cell
            deal_name = 'Сделка #' + str(deal_id)
            date_q = True
            good_time = False
            continue
    if date_q:
        try:
            date = find_date(sh.cell_value(rowx=row, colx=0))
            # date = datetime.datetime(*xlrd.xldate_as_tuple(date, book.datemode))
            # date = date.strftime("%d.%m.%y")
        except:
            print('date_q ', sh.cell_value(rowx=row, colx=0), row)
        date_q = False
        good_time = True
        continue

    if good_time:
        if cell == '' or sh.cell_value(rowx=row, colx=1) == '':
            continue
        elif 'пожалуйста' in cell.lower():
            good_time = False
        elif 'доставка' in cell.lower() or 'отправка' in cell.lower():
            product_name = cell
            price = sh.cell_value(rowx=row, colx=1)
            quantity = 1
            agent = 0
            try:
                total = float(price) * float(quantity)
            except:
                print('product_name - price ', product_name, price, 'row=', row,' |', quantity)
            code, country = find_code(contact, product_name)
            avdate, avsum = find_avdate(contact, code)
            output.loc[prod_id] = [deal_id, deal_name, direction, contact, product_name, price, quantity, agent, total,
                                   status, our_pc, total_weight, country, wh_pc, avdate, avsum, date, code]
            prod_id += 1

        else:
            product_name = cell
            price = sh.cell_value(rowx=row, colx=1)
            quantity = sh.cell_value(rowx=row, colx=3)
            agent = sh.cell_value(rowx=row, colx=6)
            try:
                total = float(price) * float(quantity)
            except:
                print('product_name price ', product_name, price, quantity, 'row=', row, 'доставка' in cell.lower())
            code, country = find_code(contact, product_name)
            avdate, avsum = find_avdate(contact, code)
            # df = pd.DataFrame({'ID': deal_id, 'Направление': direction, 'Контакт': contact, 'Товар': product_name,
            #                    'Цена за 1 шт': price, 'Количество': quantity, 'Агентское вознаграждение': agent,
            #                    'Итого': total, 'Стадия сделки': status, 'Код посылки наш': our_pc,
            #                    'Суммарный вес': total_weight, 'Страна': country, 'Код посылки склада': wh_pc,
            #                    'Дата аванса': avdate, 'Сумма аванса': avsum, 'Выдали покупку': date,
            #                    'Код покупки': code})
            output.loc[prod_id] = [deal_id, deal_name, direction, contact, product_name, price, quantity, agent, total,
                                   status, our_pc, total_weight, country, wh_pc, avdate, avsum, date, code]
            prod_id += 1

print('done')

product_name price  Трусы VS рр М 340.0  row= 1761 False
This result is True
True
Hi man
done


In [13]:
# print(pd)
print('Ошибки получения кода')
#print(code_errs)
print(len(code_errs))
for elem in code_errs:
    print(elem[0])
print('\n\n\n')

Ошибки получения кода
29
Лоншакова Елена
Лоншакова Елена
Лоншакова Елена
Обломова Елена
Рыбина Татьяна
Аня Шанскова
Аня Шанскова
Долгачева Наталья
Шпаковская Елена
Тынкасова Ольчик
Тынкасова Ольчик
Тынкасова Ольчик
Тынкасова Ольчик
Яковенко Ирина
Яковенко Ирина
Яковенко Ирина
Strogaleva Olesia
Андрейкина Елена
Сафронова Мария
Сафронова Мария
Сафронова Мария
Андрейкина Валентина
Маслова Юлька
Салошина Александра
Шанскова Аня
Романецкая Катеринка
Чугунова Вероника
Ананьева Юля
Долгачева Наталья






In [14]:
for elem in code_errs:
    print(elem[1])
print('\n\n\n')

Футболка TH рр L
Мужское поло ТН рр L
Доставка. США - 24/21
Фигурный шоколад Weibler
Термо-леггинсы жен Ernstings family черные рр L
Футболка TU CLOTHING рр 7
Футболка TU CLOTHING рр 10
Трусы VS рр S
Джемпер TU CLOTHING рр 5-6
Трусы VS рр L
Трусы VS рр L
Рубашка розовая СК рр L
Джемпер CK черный рр L
Комплект ASDA рр 4-5
Набор леггинсов ASDA рр 4-5
Платье ASDA рр 4-5
Отправка заказов 30.03 (Бонус!)
Кардиган Cortefiel рр S
Трусы VS рр М
Набор купальников ASDA рр 8-9
Набор шорт ASDA рр 8-9
Доставка. Англия -2/22
Трусы VS рр S
Отправка заказов 04.05.22 (Бонус!)
Маечка для мальчика Baykar
Отправка заказов 04.05.22
Отправка заказов 18.05.22
Отправка заказов 29.05.22
Отправка заказов 29.05.22






In [15]:
for elem in code_errs:
    print(elem[2])
print('\n\n\n')

Елена Лоншакова
Елена Лоншакова
Елена Лоншакова
Елена Обломова
Татьяна Рыбина
Шанскова Аня
Шанскова Аня
Наталья Долгачева
Елена Шпаковская
Ольчик Тынкасова
Ольчик Тынкасова
Ольчик Тынкасова
Ольчик Тынкасова
Ирина Яковенко
Ирина Яковенко
Ирина Яковенко
Olesia Strogaleva
Елена Андрейкина
Мария Сафронова
Мария Сафронова
Мария Сафронова
Валентина Андрейкина
Юлька Маслова
Александра Салошина
Аня Шанскова
Катеринка Романецкая
Вероника Чугунова
Юля Ананьева
Наталья Долгачева






In [16]:
for elem in code_errs:
    print(elem[3])
print('\n\n\n')

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None






In [17]:
print('Ошибки получения аванса')
# print(avdate_errs)
print(len(avdate_errs))
for elem in avdate_errs:
    print(elem[0])

Ошибки получения аванса
336
Кротова Татьяна
Кротова Татьяна
Лоншакова Елена
Лоншакова Елена
Лоншакова Елена
Обломова Елена
Рыбина Татьяна
Ахмеджанова Диля
Милая Наталия
Аня Шанскова
Аня Шанскова
Аня Шанскова
Аня Шанскова
Бабихина Людмила
Волкова Анна
Волкова Анна
Волкова Анна
Долгачева Наталья
Катырева Анастасия
Катырева Анастасия
Катырева Анастасия
Катырева Анастасия
Катырева Анастасия
Катырева Анастасия
Катырева Анастасия
Кудрявцева Татьяна
Кудрявцева Татьяна
Кудрявцева Татьяна
Марченко Татьяна
Шпаковская Елена
свиридюк наталья
свиридюк наталья
Иванова Ольга
Иванова Ольга
Алексеева Наталья
Алексеева Наталья
Буракова Оксана
Иванова Елена
Иванова Елена
Иванова Елена
Корчагина Александра
Корчагина Александра
Корчагина Александра
Корчагина Александра
Корчагина Александра
Корчагина Александра
Корчагина Александра
Корчагина Александра
Краснова Эльмира
Краснова Эльмира
Краснова Эльмира
Краснова Эльмира
Мищенко Ольга
Мищенко Ольга
Im Albina
Пастушина Виктория
Нилова Юлечка
Нилова Юлечка
Нило

In [18]:
for elem in avdate_errs:
    print(elem[1])

Доставка. США -2/22
Отправка заказов 9.02
None
None
None
None
None
Доставка. Англия -1/22
Сибирский кедр. Конфеты с кедровым орехом 01/22
None
None
Доставка. Англия -1/22
Отправка заказов 9.02
Доставка. США -3/22
OLD_27/11
Доставка. США -3/22
Отправка заказов 16.02
None
VS_25/11
LOCAL_02/02
LOCAL_02/02
Доставка. США -38/21
Доставка. США -1/22
Доставка. США -2/22
Доставка. США -3/22
MACYS_17/11
MACYS_17/11
MACYS_17/11
Доставка. США -3/22
None
ASDA_25/10
OLD_16-17/11
Доставка. США -2/22
Доставка. США -4/22
SAKS_26-27/11
Отправка заказов 02.03.2022
Отправка заказов 02.03.2022
LOCAL 21.12
LOCAL 21.12
LOCAL 21.12
Сибирский кедр. Конфеты с кедровым орехом 01/22
Сибирский кедр. Конфеты с кедровым орехом 01/22
Сибирский кедр. Конфеты с кедровым орехом 01/22
Сибирский кедр. Конфеты с кедровым орехом 01/22
Сибирский кедр. Конфеты с кедровым орехом 01/22
Сибирский кедр. Конфеты с кедровым орехом 01/22
Сибирский кедр. Конфеты с кедровым орехом 01/22
Отправка заказов 02.03.2022
BF_9/11
NB_24/11
UGG

In [19]:
output.to_excel('./result_last.xlsx')

In [20]:
name = 'Шанскова Аня'
name_two = ' '.join(name.split()[::-1])
print(name_two, '!', name, '!')
print('Аня Шанскова' == name_two)

Аня Шанскова ! Шанскова Аня !
True
